In [12]:
import pandas as pd
import numpy as np

In [1]:
# import Phase-1 utilities
LIGHT_IMPORT = True        # 
cands = pd.DataFrame(); uid = 1234   # placeholders for stray debug prints

%run -i "Calc_Embeddings_Model Training.ipynb"

print("df shape:", df.shape if 'df' in globals() else "df missing")
needed = ["top_k_semantic_neighbors_by_text","filter_by_orientation","filter_by_location",
    "filter_by_age","compute_component_scores","apply_weights","get_matches"]
print({n: (n in globals()) for n in needed})

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [2]:
#OkCupid scorer
LIGHT_IMPORT = True
%run -i "ok_cupid_algo.ipynb"
print("okcupid_score available?", "okcupid_score" in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [3]:
#Hinge (supervised) scorer
LIGHT_IMPORT = True
%run -i "hinge_algo.ipynb"

if "hinge_prob" in globals() and "hinge_score" not in globals():
    hinge_score = hinge_prob

print("hinge_score available?", "hinge_score" in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

,coef (standardized)
text_sim,3.135953
lifestyle_score,1.671583
essay_len,0.145017
dist_penalty,0.000000
age_diff,-0.049669


,coef (standardized)
age_diff,-0.049669
dist_penalty,0.000000
essay_len,0.145017
lifestyle_score,1.671583
text_sim,3.135953


Start pool: 5000 (max_candidates=5000)
After orientation: 1561
Unfiltered: 1561 | After location filter (≤50 km): 1552
After location (≤50 km): 1552
After age rule: 1081
hinge_score available? True


In [21]:
# Create harmless defaults so demo code won't crash if it runs
k = 0
ranked = pd.DataFrame()
suggested = pd.DataFrame()
tinder_suggestions_df = pd.DataFrame()

def add_reasons_simple(df_in):
    return df_in

In [44]:
#Tinder scorer
LIGHT_IMPORT = True
%run -i "tinder_algo.ipynb"
print("tinder_score available?", "tinder_score" in globals())
print("tinder_suggestions available?", "tinder_suggestions" in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,10833,0.798922,0.699155,1.0,1.00,1.0,30.700929,"el sobrante, california",33,m,straight,suggested
1,54851,0.797583,0.698112,1.0,1.00,1.0,23.920071,"san francisco, california",27,m,straight,suggested
2,48244,0.796747,0.695700,1.0,1.00,1.0,19.197039,"berkeley, california",23,m,straight,suggested
3,11226,0.792767,0.689729,1.0,1.00,1.0,23.920071,"san francisco, california",31,m,straight,suggested
4,53171,0.789856,0.685749,1.0,1.00,1.0,28.113170,"daly city, california",26,m,straight,suggested
5,8379,0.787841,0.682148,1.0,1.00,1.0,16.481152,"emeryville, california",36,m,straight,suggested
6,46695,0.781504,0.726020,1.0,0.75,1.0,31.203141,"palo alto, california",31,m,straight,suggested
7,57727,0.774239,0.661552,1.0,1.00,1.0,29.764458,"san mateo, california",22,m,straight,suggested
8,13060,0.773779,0.661054,1.0,1.00,1.0,23.920071,"san francisco, california",40,m,straight,suggested
9,14484,0.773126,0.659883,1.0,1.00,1.0,23.920071,"san francisco, california",26,m,straight,suggested


tinder_score available? True
tinder_suggestions available? True


In [49]:
# 1) If the name was shadowed by a DataFrame, delete that binding
import pandas as pd, inspect, types

if "tinder_suggestions" in globals() and isinstance(tinder_suggestions, pd.DataFrame):
    del tinder_suggestions  # remove the DF that's blocking the function name

# 2) Re-run the tinder notebook so the function is defined again
%run -i "tinder_algo.ipynb"

# 3) Sanity check + make a SAFE alias so we never shadow it again
assert callable(tinder_suggestions), "tinder_suggestions is still not a function — check the tinder notebook."
tinder_suggestions_fn = tinder_suggestions  # safe alias to call later

import inspect
print("tinder_suggestions signature:", inspect.signature(tinder_suggestions_fn))

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,10833,0.798922,0.699155,1.0,1.00,1.0,30.700929,"el sobrante, california",33,m,straight,suggested
1,54851,0.797583,0.698112,1.0,1.00,1.0,23.920071,"san francisco, california",27,m,straight,suggested
2,48244,0.796747,0.695700,1.0,1.00,1.0,19.197039,"berkeley, california",23,m,straight,suggested
3,11226,0.792767,0.689729,1.0,1.00,1.0,23.920071,"san francisco, california",31,m,straight,suggested
4,53171,0.789856,0.685749,1.0,1.00,1.0,28.113170,"daly city, california",26,m,straight,suggested
5,8379,0.787841,0.682148,1.0,1.00,1.0,16.481152,"emeryville, california",36,m,straight,suggested
6,46695,0.781504,0.726020,1.0,0.75,1.0,31.203141,"palo alto, california",31,m,straight,suggested
7,57727,0.774239,0.661552,1.0,1.00,1.0,29.764458,"san mateo, california",22,m,straight,suggested
8,13060,0.773779,0.661054,1.0,1.00,1.0,23.920071,"san francisco, california",40,m,straight,suggested
9,14484,0.773126,0.659883,1.0,1.00,1.0,23.920071,"san francisco, california",26,m,straight,suggested


tinder_suggestions signature: (user_id, random_sample=10, pool_k=500, max_km=50, drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True, weights=None)


In [50]:
# Bumble + eHarmony scorer
LIGHT_IMPORT = True
%run -i "bumble+eHarmony algo.ipynb"
print("bumble_rank_basic available?", "bumble_ranking" in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
User 42 is sex='M' — per Bumble rules, he cannot initiate. Skipping openers.
Start pool: 5000 (max_candidates=5000)
After orientation: 2401
Unfiltered: 2401 | After location filter (≤20 km): 1863
After location (≤20 km): 1863
After age rule: 1718

Bumble Score = 0.7705
User id = 1540      age = 33      location = oakland, california
bio text =
i'm not good at elevator pitches but here goes: my favorite color is yellow. i think i'm right most of the time but i love being proved wrong. my friends ...
Suggested pick up lines =
 1. If you could only drink soup from a liquor bottle, what flavor would you choose? 🍲🥃
 2. Yellow is the color of sunshine—what’s your best adventure under its glow? ☀️🚴‍♀️
 3. Trivial Pursuit champion? Let’s settle this with a friendly game and some burritos! 🌯🎲
Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤20 km): 522
After location (≤20 k

In [32]:
# --- OKCupid: score ---
okcupid = okcupid_score(
    user_id=15,
    k=10,
    weights={"text":0.70, "age":0.15, "life":0.10, "loc":0.05},
    pool_k=5000,
    max_km=45,
    allow_missing=True,
    drinks_strict=False, smokes_strict=True, drugs_strict=True,
    hard_filters={"diet": False, "drinks": False, "smokes": True, "drugs": True})
okcupid.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2005
Unfiltered: 2005 | After location filter (≤45 km): 1941
After location (≤45 km): 1941
After age rule: 1478
Smokes | before: 1478 → after: 1277 (strict=True)
After smokes (strict=True): 1277
Drugs  | before: 1277 → after: 1277 (strict=True)
After drugs (strict=True): 1277


,user_id,compat_pct,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,28566,77.8,0.682167,1.0,1.0,1.0,0.000000,"san francisco, california",30,m,straight
1,49783,77.7,0.681057,1.0,1.0,1.0,13.300941,"oakland, california",42,m,straight
2,58244,77.1,0.672608,1.0,1.0,1.0,26.260308,"moraga, california",48,m,straight
3,20973,76.6,0.665855,1.0,1.0,1.0,32.302391,"san mateo, california",43,m,straight
4,33449,76.6,0.665802,1.0,1.0,1.0,0.000000,"san francisco, california",50,m,straight
5,37557,76.5,0.664575,1.0,1.0,1.0,21.343016,"san pablo, california",27,m,straight
6,57916,76.4,0.663267,1.0,1.0,1.0,17.174141,"san bruno, california",37,m,straight
7,3938,76.4,0.662820,1.0,1.0,1.0,10.929292,"daly city, california",51,m,straight
8,58552,76.3,0.662118,1.0,1.0,1.0,43.696864,"palo alto, california",49,m,straight
9,5568,76.3,0.661580,1.0,1.0,1.0,16.026195,"albany, california",48,m,straight


In [34]:
# --- Hinge: probability model ---
# Hinge inference only (pool built internally by hinge_prob)
hinge = hinge_prob(
    user_id=15,
    k=10,
    model=hinge_model,
    pool_k=5000,
    max_km=45)
hinge.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2005
Unfiltered: 2005 | After location filter (≤45 km): 1941
After location (≤45 km): 1941
After age rule: 1478


,user_id,age,sex,orientation,location,text_sim,age_score,age_diff,lifestyle_score,loc_score,dist_penalty,distance_km,essay_len,prob_mutual_like,feature_breakdown
0,28566,30.0,m,straight,"san francisco, california",0.682167,1.0,9,1.0,1.0,0.0,0.000000,1740,0.998035,"text=0.68, life=1.00, age=1, loc=1.00, Δage=9,..."
1,49783,42.0,m,straight,"oakland, california",0.681057,1.0,3,1.0,1.0,0.0,13.300941,1062,0.997894,"text=0.68, life=1.00, age=1, loc=1.00, Δage=3,..."
2,58244,48.0,m,straight,"moraga, california",0.672608,1.0,9,1.0,1.0,0.0,26.260308,2489,0.997040,"text=0.67, life=1.00, age=1, loc=1.00, Δage=9,..."
3,14899,45.0,m,straight,"san francisco, california",0.667275,1.0,6,1.0,1.0,0.0,0.000000,620,0.995373,"text=0.67, life=1.00, age=1, loc=1.00, Δage=6,..."
4,20973,43.0,m,straight,"san mateo, california",0.665855,1.0,4,1.0,1.0,0.0,32.302391,713,0.995174,"text=0.67, life=1.00, age=1, loc=1.00, Δage=4,..."
5,33449,50.0,m,straight,"san francisco, california",0.665802,1.0,11,1.0,1.0,0.0,0.000000,953,0.994933,"text=0.67, life=1.00, age=1, loc=1.00, Δage=11..."
6,37557,27.0,m,straight,"san pablo, california",0.664575,1.0,12,1.0,1.0,0.0,21.343016,876,0.994502,"text=0.66, life=1.00, age=1, loc=1.00, Δage=12..."
7,57916,37.0,m,straight,"san bruno, california",0.663267,1.0,2,1.0,1.0,0.0,17.174141,445,0.994442,"text=0.66, life=1.00, age=1, loc=1.00, Δage=2,..."
8,50919,30.0,m,straight,"san francisco, california",0.656761,1.0,9,1.0,1.0,0.0,0.000000,4060,0.994374,"text=0.66, life=1.00, age=1, loc=1.00, Δage=9,..."
9,3938,51.0,m,straight,"daly city, california",0.662820,1.0,12,1.0,1.0,0.0,10.929292,1272,0.994238,"text=0.66, life=1.00, age=1, loc=1.00, Δage=12..."


In [38]:
# --- Bumble: ranked output ---
bumble = bumble_ranking(
    user_id=15,
    k=10,         
    pool_k=5000,    
    max_km=45)
bumble.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2005
Unfiltered: 2005 | After location filter (≤45 km): 1941
After location (≤45 km): 1941
After age rule: 1478


,user_id,bumble_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,35029,0.845005,0.690010,1.0,0.666667,1.0,0.000000,"san francisco, california",33,m,straight
1,28566,0.841084,0.682167,1.0,1.000000,1.0,0.000000,"san francisco, california",30,m,straight
2,49783,0.840528,0.681057,1.0,1.000000,1.0,13.300941,"oakland, california",42,m,straight
3,58244,0.836304,0.672608,1.0,1.000000,1.0,26.260308,"moraga, california",48,m,straight
4,20571,0.834426,0.668853,1.0,0.666667,1.0,32.302391,"san mateo, california",37,m,straight
5,14899,0.833637,0.667275,1.0,1.000000,1.0,0.000000,"san francisco, california",45,m,straight
6,20973,0.832927,0.665855,1.0,1.000000,1.0,32.302391,"san mateo, california",43,m,straight
7,33449,0.832901,0.665802,1.0,1.000000,1.0,0.000000,"san francisco, california",50,m,straight
8,37557,0.832287,0.664575,1.0,1.000000,1.0,21.343016,"san pablo, california",27,m,straight
9,38875,0.832230,0.664460,1.0,0.666667,1.0,13.300941,"oakland, california",36,m,straight


In [58]:
# --- Build recency_norm exactly like in tinder_algo.ipynb ---
import pandas as pd

# Take only the first 10 characters (YYYY-MM-DD)
df["last_online_clean"] = df["last_online"].astype(str).str.slice(0, 10)

# Parse as date
df["last_online_dt"] = pd.to_datetime(df["last_online_clean"], errors="coerce", utc=True)

# Now build recency features
df["recency_rank"] = df["last_online_dt"].rank(ascending=True)  # oldest = 1
min_dt, max_dt = df["last_online_dt"].min(), df["last_online_dt"].max()
if pd.notna(min_dt) and pd.notna(max_dt) and max_dt > min_dt:
    span = (max_dt - min_dt).total_seconds()
    df["recency_norm"] = ((df["last_online_dt"] - min_dt).dt.total_seconds() / span).fillna(0.0)
else:
    df["recency_norm"] = 0.0

In [61]:
# --- Tinder ranked + suggestions  ---
ranked = tinder_score(
    user_id=15,
    k=10,
    pool_k=5000,
    max_km=45,
    drinks_strict=False,
    smokes_strict=True,
    drugs_strict=True,
    allow_missing=True
).assign(source="ranked")

suggested = tinder_suggestions(
    user_id=15,
    random_sample=20,
    pool_k=5000,
    max_km=45,
    drinks_strict=False,
    smokes_strict=True,
    drugs_strict=True,
    allow_missing=True,
    weights={"text":0.70,"age":0.15,"life":0.10,"loc":0.05})

tinder = pd.concat([ranked.head(7), suggested.head(3)], ignore_index=True).drop_duplicates("user_id", keep="first")

display(tinder.head(10))

Start pool: 5000 (max_candidates=5000)
After orientation: 2005
Unfiltered: 2005 | After location filter (≤45 km): 1941
After location (≤45 km): 1941
After age rule: 1478
Start pool: 5000 (max_candidates=5000)
After orientation: 2005
Unfiltered: 2005 | After location filter (≤45 km): 1941
After location (≤45 km): 1941
After age rule: 1478


,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,28566,0.797497,0.682167,1.0,1.0,1.0,0.000000,"san francisco, california",30,m,straight,ranked
1,49783,0.796791,0.681057,1.0,1.0,1.0,13.300941,"oakland, california",42,m,straight,ranked
2,58244,0.791414,0.672608,1.0,1.0,1.0,26.260308,"moraga, california",48,m,straight,ranked
3,20973,0.787116,0.665855,1.0,1.0,1.0,32.302391,"san mateo, california",43,m,straight,ranked
4,37557,0.786056,0.664575,1.0,1.0,1.0,21.343016,"san pablo, california",27,m,straight,ranked
5,3938,0.784940,0.662820,1.0,1.0,1.0,10.929292,"daly city, california",51,m,straight,ranked
6,57916,0.784487,0.663267,1.0,1.0,1.0,17.174141,"san bruno, california",37,m,straight,ranked
7,10638,0.748154,0.622617,1.0,1.0,1.0,32.302391,"san mateo, california",30,m,straight,suggested
8,48102,0.746065,0.619870,1.0,1.0,1.0,0.000000,"san francisco, california",34,m,straight,suggested
9,39405,0.730587,0.596074,1.0,1.0,1.0,21.350781,"pacifica, california",42,m,straight,suggested
